### Prometheus Beispiel

![](https://prometheus-operator.dev/img/architecture.png)

Quelle: [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/)

- - -

Prometheus liefert ein Beispiel eines überwachten Services.

Dieser ist in diesem [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/) beschrieben.

Wir starten das Beispiel bzw. den Microservice und erstellen gleichzeitig einen `ServiceMonitor`

In [ ]:
%%bash
kubectl create namespace mon-app

cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice   
spec:
  replicas: 3
  selector:
    matchLabels:
      app: example-app
  template:
    metadata:
      labels:
        app: example-app
        tier: microservice        
    spec:
      containers:
      - name: example-app
        image: fabxc/instrumented_app
        ports:
        - name: web
          containerPort: 8080
---
kind: Service
apiVersion: v1
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice    
spec:
  selector:
    app: example-app
  ports:
  - name: web
    port: 8080
  type:
    LoadBalancer
---    
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
%EOF%


Und kontrollieren ob alle Pods gestartet wurden

In [ ]:
! kubectl get all,servicemonitors --namespace mon-app -l tier=microservice

Ist dies der Fall, können wir die Metrics Daten abfragen

In [ ]:
! curl example-app.mon-app:8080/metrics

- - -
# Bereitstellen von Prometheus 

Die erstellten Microservices können wir nun mit Prometheus überwachen.

Dazu brauchen wir die entsprechenden Rechte:   

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: mon-app
%EOF%


Und eine eigene Prometheus Instanz.

Mittels 

      alerting:  
        alertmanagers:
        - apiVersion: v2
          name: alertmanager-example
          namespace: mon-app
          pathPrefix: /
          port: web
          
stellen wir die Verbindung zum Alertmanager her.          

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  alerting:  
    alertmanagers:
    - apiVersion: v2
      name: alertmanager-example
      namespace: mon-app
      pathPrefix: /
      port: web
  ruleSelector:
    matchLabels:
      role: alert-rules
      prometheus: prometheus      
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%

Im User Interface können wir dann die überwachten Targets und deren Status anschauen.

Dazu ist als Query `up` einzugeben oder via Pulldown -> Status -> Targets die überwachten Microservices anzuzeigen.

Ausserdem können wir alle Werte vom Befehl `curl example-app.mon-app:8080/metrics`, ebenfall abfragen.

In [ ]:
! echo "Prometheus UI  : http://$(cat ~/work/server-ip):$(kubectl get service --namespace mon-app prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

---

# [Bereitstellung Alertmanager](https://prometheus-operator.dev/docs/user-guides/alerting/)

Der Prometheus-Operator führt eine AlertmanagerRessource ein, die es Benutzern ermöglicht, einen Alertmanager-Cluster deklarativ zu beschreiben. Um einen Alertmanager-Cluster erfolgreich bereitzustellen, ist es wichtig, die Interaktion zwischen Prometheus und dem Alertmanager zu verstehen. 

Der Alertmanager wird verwendet für:
* Von Prometheus empfangene Warnungen deduplizieren.
* Warnungen stummschalten.
* Weiterleiten von Benachrichtigungen an verschiedene Endpunkte (PagerDuty, OpsGenie, Mail, Chat, …).



In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1
kind: Alertmanager
metadata:
  name: example
spec:
  replicas: 1
  alertmanagerConfigSelector:
    matchLabels:
      alertmanagerConfig: example
---
apiVersion: v1
kind: Service
metadata:
  name: alertmanager-example
spec:
  type: NodePort
  ports:
  - name: web
    port: 9093
    protocol: TCP
    targetPort: web
  selector:
    alertmanager: example
%EOF%

Mittels `AlertmanagerConfig` können wir dem Alertmanager mitteilen wie auf Alerts zu reagieren ist.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1alpha1
kind: AlertmanagerConfig
metadata:
  name: config-example
  labels:
    alertmanagerConfig: example
spec:
  route:
    groupBy: ['job']
    groupWait: 30s
    groupInterval: 5m
    repeatInterval: 12h
    receiver: 'webhook'
  receivers:
  - name: 'webhook'
    webhookConfigs:
    - url: 'http://example.com/'
%EOF%

Und mittels `PrometheusRule` wird in Prometheus die entsprechende Regel aktiviert

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  creationTimestamp: null
  labels:
    prometheus: prometheus
    role: alert-rules    
  name: prometheus-example-rules
spec:
  groups:
  - name: example-app
    rules:
    - alert: ExampleAlert
      expr: vector(1)
%EOF%


## Kontrolle

Im Prometheus UI sollte unter den Pulldowns
* Status -> Runtime Information & Build Information ein Alertmanager Endpoint vorhanden sein.
* Status -> Drei Instanzen der Example-App
* Status -> Rules die Example-App Rule
Und bei `Alerts` eine Benachrichtigung.

Im Alertmanager UI ist ebenfalls eine Benachrichtung ersichtlich.

In [ ]:
! echo "Prometheus   UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace mon-app prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"
! echo "Alertmanager UI: http://$(cat ~/work/server-ip):$(kubectl -n mon-app get service alertmanager-example -o=jsonpath='{ .spec.ports[0].nodePort }')"

---
### Aufräumen

In [ ]:
! kubectl delete namespace mon-app